In [1]:
import os
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from keybert import KeyBERT
from PyDictionary import PyDictionary
import yake
from keybert import KeyBERT

In [2]:
nltk.download("stopwords")
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
nlp = spacy.load('en_core_web_sm')

In [11]:
corpus = pd.read_csv("/Users/alex/Desktop/ВШЭ/Python/Project/df-corpus.csv", sep='\t')

Ключевые слова

In [10]:
def yake_keywords(text):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 7
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text.lower())
    keywords1 = []
    for kw in keywords:
        keywords1.append(kw[0])
    lemmatized_keywords = ' '.join(keywords1)
    doc = nlp(lemmatized_keywords)
    lemmatized_keywords = [i.lemma_ for i in doc]
    lemmatized_keywords = list(set(lemmatized_keywords))     
    return lemmatized_keywords

Ключевые слова со словарными значениями

In [112]:
def keywords_with_definitions(text):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None)
    keybert_keywords = [i[0] for i in keywords]
    yake = yake_keywords(text)
    all_keywords = keybert_keywords + yake
    lemmatized_keywords = ' '.join(all_keywords)
    doc = nlp(lemmatized_keywords)
    lemmatized_keywords = [i.lemma_ for i in doc]
    lemmatized_keywords = list(set(lemmatized_keywords))
    dictionary = PyDictionary()
    definitions = [dictionary.meaning(i, disable_errors=True) for i in lemmatized_keywords]
    words_with_definitions = dict(zip(lemmatized_keywords,definitions))
    strings = []
    for key,item in words_with_definitions.items():
        strings.append("{} - {}".format(key.upper(), item))
    result = "\n".join(strings)
    return result

Саммари

In [190]:
def summary(text):
    docx = nlp(text)
    mytokens = [token.text for token in docx]
    word_frequencies = {}
    for word in docx:
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
    maximum_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    sentence_list = [sentence for sentence in docx.sents]
    sentence_scores = {}
    for sent in sentence_list:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    list_of_sentences = list(sentence_scores)
    summary = ""
    for i in list_of_sentences:
        summary += str(i)+ " " 
    return summary

In [ ]:
corpus['keywords with definitions'] = corpus['texts'].apply(keywords_with_definitions)

In [ ]:
corpus['summary'] = corpus['texts'].apply(summary)

In [3]:
display(corpus)

NameError: name 'corpus' is not defined

Именнованные сущности NER

In [64]:
corpus = pd.read_csv('Final corpus.csv')

In [65]:
nlp = spacy.load('en_core_web_lg')

In [74]:
exemple = corpus['texts'][183]

In [89]:
# # Если понадобится список именованных сущностей: 
# doc = nlp(exemple)
# for entity in doc.ents:
#     print(f"{entity.text} ({entity.label_})")

textacy.extract.basics.entities(doc, *, include_types: Optional[str | Collection[str]] = None, exclude_types: Optional[str | Collection[str]] = None, drop_determiners: bool = True, min_freq: int = 1) 
Parameters
doclike –
include_types – Remove entities whose type IS NOT in this param; if “NUMERIC”, all numeric entity types (“DATE”, “MONEY”, “ORDINAL”, etc.) are included
exclude_types – Remove entities whose type IS in this param; if “NUMERIC”, all numeric entity types (“DATE”, “MONEY”, “ORDINAL”, etc.) are excluded
drop_determiners –
Remove leading determiners (e.g. “the”) from entities (e.g. “the United States” => “United States”).
min_freq – Remove entities that occur in doclike fewer than min_freq times

In [90]:
textacy.extract.basics.entities(doc, exclude_types = 'ORDINAL')

<generator object entities at 0x7f9d087ff350>


Для именованных сущностей найти двуграммы:

In [ ]:
textacy.extract.basics.ngrams(doc, (2,3), 

textacy.extract.basics.ngrams(doclike: types.DocLike, n: int | Collection[int], *, filter_stops: bool = True, filter_punct: bool = True, filter_nums: bool = False, include_pos: Optional[str | Collection[str]] = None, exclude_pos: Optional[str | Collection[str]] = None, min_freq: int = 1) → Iterable[Span]

Parameters
doclike –
n – Number of tokens included per n-gram; for example, 2 yields bigrams and 3 yields trigrams. If multiple values are specified, then the collections of n-grams are concatenated together; for example, (2, 3) yields bigrams and then trigrams.
filter_stops – If True, remove ngrams that start or end with a stop word.
filter_punct – If True, remove ngrams that contain any punctuation-only tokens.
filter_nums – If True, remove ngrams that contain any numbers or number-like tokens (e.g. 10, ‘ten’).
include_pos – Remove ngrams if any constituent tokens’ part-of-speech tags ARE NOT included in this param.
exclude_pos – Remove ngrams if any constituent tokens’ part-of-speech tags ARE included in this param.
min_freq – Remove ngrams that occur in doclike fewer than min_freq times

Визуализация для английских текстов:

In [76]:
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(exemple)
displacy.serve(doc, style="ent")

OSError: [Errno 48] Address already in use

In [93]:
import spacy
import textacy.extract
doc = nlp(exemple)
statements = textacy.extract.semistructured_statements(doc, entity='', cue='')


for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

Извлечения часто упоминаемых фрагментов из документа:

In [91]:
doc = nlp(exemple)

noun_chunks = textacy.extract.noun_chunks(doc, min_freq=2)

noun_chunks = map(str, noun_chunks)
noun_chunks = map(str.lower, noun_chunks)

for noun_chunk in set(noun_chunks):
    if len(noun_chunk.split(" ")) > 1:
        print(noun_chunk)

cork oak
screw caps


n-grams:

In [82]:
# import re
# from nltk.util import ngrams

# exemple = exemple.lower()
# exemple = re.sub(r'[^a-zA-Z0-9\s]', ' ', exemple)
# tokens = [token for token in exemple.split(" ") if token != ""]
# output = list(ngrams(tokens, 5))
# print(output)

[('cork\n\ncork', 'the', 'thick', 'bark', 'of'), ('the', 'thick', 'bark', 'of', 'the'), ('thick', 'bark', 'of', 'the', 'cork'), ('bark', 'of', 'the', 'cork', 'oak'), ('of', 'the', 'cork', 'oak', 'tree'), ('the', 'cork', 'oak', 'tree', 'quercus'), ('cork', 'oak', 'tree', 'quercus', 'suber'), ('oak', 'tree', 'quercus', 'suber', 'is'), ('tree', 'quercus', 'suber', 'is', 'a'), ('quercus', 'suber', 'is', 'a', 'remarkable'), ('suber', 'is', 'a', 'remarkable', 'material'), ('is', 'a', 'remarkable', 'material', 'it'), ('a', 'remarkable', 'material', 'it', 'is'), ('remarkable', 'material', 'it', 'is', 'tough'), ('material', 'it', 'is', 'tough', 'elastic'), ('it', 'is', 'tough', 'elastic', 'buoyant'), ('is', 'tough', 'elastic', 'buoyant', 'and'), ('tough', 'elastic', 'buoyant', 'and', 'fire'), ('elastic', 'buoyant', 'and', 'fire', 'resistant'), ('buoyant', 'and', 'fire', 'resistant', 'and'), ('and', 'fire', 'resistant', 'and', 'suitable'), ('fire', 'resistant', 'and', 'suitable', 'for'), ('resis

Документация на используемые библиотеки:
spaCy - https://spacy.io/usage/linguistic-features
улучшить spaCy: https://github.com/huggingface/neuralcoref
textacy - https://textacy.readthedocs.io/en/0.12.0/api_reference/extract.html#textacy.extract.triples.semistructured_statements